# Inversion geoelektrischer Profilmessungen mit `pyGIMLi` 

Das Ziel der Inversion besteht in der Rekonstruktion der räumlichen Verteilung der spezifischen Widerstände im Untergrund.
Grundlage bilden die gemessenen Daten, in der Regel sind das Ohmsche Widerstände $R(L)$ in $\Omega$ oder scheinbare spezifische Widerstände $\rho_s(L)$ in $\Omega\cdot m$.
Sämtliche Messwerte sind abhängig vom Ort und der geoelektrischen Elektrodenkonfiguration (hier durch $L$ gekennzeichnet).

### Installation

Bei `pyGIMLi` handelt es sich um eine Python-Bibliothek.
Wir setzen voraus, dass eine Python-Umgebung bereits installiert ist.
Am einfachsten funktioniert [*anaconda*](https://www.anaconda.com/products/individual).

Zunächst muss `pyGIMLi` installiert werden, damit unter Python darauf zugegriffen werden kann.
Das geschieht mit `conda`:
```shell
conda install -c gimli -c conda-forge pygimli=1.2.2
```

### Vorbereitung

Wir importieren die beiden Bibliotheken `pygimli` und `numpy`.

In [ ]:
import pygimli as pg
import numpy as np
from pygimli.physics.ert import ERTManager

# Das Datenformat

`pyGIMLi` erwartet die geoelektrischen Messwerte in einem speziellen Dateiformat, dem *Unified Data Format* [(Beschreibung auf der Projektseite von BERT)](https://gitlab.com/resistivity-net/bert#the-unified-data-format).
Es handelt sich um ASCII-Dateien, welche mit allen gängigen Texteditoren editiert werden können.

Eine solche Messwertdatei enthält folgende Abschnitte:
- Anzahl der Elektroden sowie Koordinaten (Lage und Höhe) der Elektroden mit impliziter Nummerierung
- Anzahl der Messwerte sowie die sich jeweils aus der Kombination von zwei bis vier Elektroden ergebenden Daten

Das Beispiel zeigt einen kleinen Datensatz mit 6 Elektroden und 6 Einzelmessungen in der Konfiguration *Dipol-Dipol* (erkennbar an der Reihenfolge *a b m n* der Elektroden und Sonden).
Die 5. Spalte des Datenblocks enthält den scheinbaren spezifischen Widerstand (`rhoa`) in $\Omega\cdot m$.

```text
6# Number of electrodes
# x z position for each electrode
0     0
1     0
2     0 # loose ground
3     0
4     0
5     0
6# Number of data
# a b m n rhoa
1   2   3   4  231.2
2   3   4   5  256.7
3   4   5   6  312.8 
1   2   4   5  12.1 # possibly an outlier
2   3   5   6  199.7
1   2   5   6  246.2
```

## Inversion eines echten Datensatzes
Wir verwenden für dieses Beispiel Daten einer Messung auf dem Campusgelände der TU Bergakademie Freiberg. Der "Erzgang" ist aus dem Grundlagenpraktikum bereits bekannt (Versuch *Elektromagnetische Kartierung*).

Die Profilmessung wurde mit den Konfigurationen *Wenner*, *Dipol-Dipol* und *Pol-Dipol* mit 21 Elektroden durchgeführt.

Der Dipol-Dipol-Datensatz besteht aus 93, der Pol-Dipol-Datensatz aus 99 und der Wenner-Datensatz aus 63 Einzelmessungen.

Die Python-Objekte `dipoldipol`, `wenner` und `poldipol` enthalten sämtliche gemessenen Daten:

In [ ]:
wenner = pg.load("erzw_2021.dat")
print(wenner)

In [ ]:
dipoldipol = pg.load("erzd_2021.dat")
print(dipoldipol)

In [ ]:
poldipol = pg.load("erzp_2021.dat")
print(poldipol)

Die Datei `all_2021.dat` enthält sämtliche 255 Einzelmessungen als Kombination.

In [ ]:
all = pg.load("all_2021.dat")
print(all)

In der `pyGIMLi`-Methode `ERTManager()` sind alle Funktionen zur Inversion von Geoelektrikdaten (engl. *Electrical Resistivity Tomography*, ERT) implementiert.

Die Objekte `ertwenner` bzw. `ertpoldipol` werden u.a. die Modellantwort und die Modellparameter aufnehmen.

In [ ]:
ertwenner = ERTManager()
ertpoldipol = ERTManager()
ertdipoldipol = ERTManager()

In [ ]:
ert_all = ERTManager()

Da unsere Datensätze keine Messfehler enthalten, schätzen wir plausible absolute und relative Datenfehler ab.
Absolute Fehler sind in in $\Omega\cdot m$ angegeben.

In [ ]:
wenner['err'] = ertwenner.estimateError(wenner, absoluteError=0.1, relativeError=0.03)
dipoldipol['err'] = ertdipoldipol.estimateError(dipoldipol, absoluteError=0.1, relativeError=0.03)
poldipol['err'] = ertpoldipol.estimateError(poldipol, absoluteError=0.1, relativeError=0.03)

In [ ]:
all['err'] = ert_all.estimateError(all, absoluteError=0.1, relativeError=0.03)

Die folgende Abbildung zeigt eine *Pseudosektion* der **scheinbaren spezifischen Widerstände** $\rho_s$.
Die Trapezform kommt durch die willkürliche Wahl des horizontalen Bezugspunktes im Profilverlauf für die Wenner- bzw. Pol-Dipol-Anordnung zustande.
Die Ordinaten in den beiden folgenden Abbildungen stellen den Elektrodenabstand bei Wenner bzw. die Separation zwischen Strompol und Spannungsdipol bei Pol-Dipol dar.
**Es handelt sich nicht um eine Tiefenachse!**

In [ ]:
ertwenner.showData(wenner, cMap="RdBu_r");

In [ ]:
ertdipoldipol.showData(dipoldipol, cMap="RdBu_r");

In [ ]:
ertpoldipol.showData(poldipol, cMap="RdBu_r");

In [ ]:
ert_all.showData(all, cMap="RdBu_r")

### Dateninversion

Die Methode `ert.invert` führt die Inversion der Daten (`data`) für einen festgelegten Regularisierungsparameter $\lambda$ (`lam=10`) sowie eine maximale Modelltiefe von `paraDepth=10` Metern und einer maximalen Dreiecksgröße der Finiten Elemente von `paraMaxCellSize=1` $m^2$ durch.

Die Resultate der Inversionsrechnung -- die Verteilung der spezifischen Widerstände im Untergrund und die Modellantwort -- werden im Objekt `mod` gespeichert.

Dem Parameter `lam` wird der Wert des gewählten *Regularisierungsparameters* $\lambda$ zugeordnet.

Bei zu niedrigen Werten von $\lambda$ äußert sich eine ggf. vorliegende Singularität des Kleinste-Quadrate-Problems durch stark schwankende Werte in der Widerstandsverteilung im Untergrund.
Im umgekehrten Fall wird bei zu großem $\lambda$ die Widerstandsverteilung sehr glatt.

Hier können wir ausprobieren, welchen Einfluss der Regularisierungsparameter auf die Lösung unseres Problems hat.

In [ ]:
lam = 10

In [ ]:
modwenner = ertwenner.invert(wenner, lam=lam, paraDepth=10, paraMaxCellSize=0.5, verbose=False);

In [ ]:
moddipoldipol = ertdipoldipol.invert(dipoldipol, lam=lam, paraDepth=10, paraMaxCellSize=0.5, verbose=False);

In [ ]:
modpoldipol = ertpoldipol.invert(poldipol, lam=lam, paraDepth=10, paraMaxCellSize=0.5, verbose=False);

In [ ]:
mod_all = ert_all.invert(all, lam=lam, paraDepth=10, paraMaxCellSize=0.5, verbose=False);

Die folgenden Abbildungen veranschaulichen den Zusammenhang zwischen den Modellparametern ($\rho(\mathbf r)$) sowie den gemessenen scheinbaren spezifischen Widerständen (`Data`) und der Modellantwort (`Response`).

In [ ]:
ertwenner.showResultAndFit(cMap="RdBu_r");

In [ ]:
ertdipoldipol.showResultAndFit(cMap="RdBu_r");

In [ ]:
ertpoldipol.showResultAndFit(cMap="RdBu_r");

In [ ]:
ert_all.showResultAndFit(cMap="RdBu_r");

Das Modell kann hier etwas genauer betrachtet werden.
Es fällt auf, dass im Bereich der Profilmeter 10-15 eine steil einfallende Zone mit geringerem spezifischen Widerstand auftritt.
Das Umgebungsgestein ist Freiberger Gneis.
Bei der Störung handelt es sich um eine tonführende Verwitterungszone.

In [ ]:
ertwenner.showModel(modwenner, cMap="RdBu_r");

In [ ]:
ertdipoldipol.showModel(moddipoldipol, cMap="RdBu_r");

In [ ]:
ertpoldipol.showModel(modpoldipol, cMap="RdBu_r");

In [ ]:
ert_all.showModel(mod_all, cMap="RdBu_r");

Von Interesse ist die Anpassungsgüte, die wir als *misfit* bezeichnen.
Dabei wird der relative Fehler zwischen Daten und Modellantwort gebildet.
Aufschlussreich ist eine graphische Darstellung dieses Fehlers:

In [ ]:
misfitwe = ertwenner.inv.response / wenner['rhoa'] * 100 - 100
me = np.max(np.abs(misfitwe))
pg.show(wenner, misfitwe, cMap="RdBu_r", cMin=-me, cMax=me, label="misfit (%)");

In [ ]:
misfitdd = ertdipoldipol.inv.response / dipoldipol['rhoa'] * 100 - 100
me = np.max(np.abs(misfitdd))
pg.show(dipoldipol, misfitdd, cMap="RdBu_r", cMin=-me, cMax=me, label="misfit (%)");

In [ ]:
misfitpd = ertpoldipol.inv.response / poldipol['rhoa'] * 100 - 100
me = np.max(np.abs(misfitpd))
pg.show(poldipol, misfitpd, cMap="RdBu_r", cMin=-me, cMax=me, label="misfit (%)");

In [ ]:
misfit_all = ert_all.inv.response / all['rhoa'] * 100 - 100
me = np.max(np.abs(misfit_all))
pg.show(all, misfit_all, cMap="RdBu_r", cMin=-me, cMax=me, label="misfit (%)");

Der globale Anpassungsfehler wird mit der $\chi^2$-Statistik beschrieben und beträgt für Wenner

In [ ]:
ertwenner.inv.chi2()

für Dipol-Dipol

In [ ]:
ertdipoldipol.inv.chi2()

sowie für Pol-Dipol

In [ ]:
ertpoldipol.inv.chi2()

In [ ]:
ert_all.inv.chi2()

## Diskussion

Wir haben gesehen, dass das Inversionsergebnis durch die Wahl verschiedener Parameter beeinflusst werden kann. Vor allem die Datenanpassung und Modelleigenschaften sind betroffen.

- Welcher Parameter steuert die Rauhigkeit des Modells?
- Welche Konfiguration löst die Störungszone besser auf?